In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
# Reading instensities template

In [4]:
intensities_template = pd.read_csv("intensities_template.csv")

In [5]:
# Reading pixels files to calculte volumes

In [6]:
header = ["PIXEL_COUNT"]
pixels_count_volume = pd.read_csv("SMM_s1_ABA_histogram_total.csv", names = header)
pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume)))  
pixels_count_volume_in_tissue = pd.read_csv("SMM_s1_ABA_histogram_in_tissue_total.csv", names = header)
pixels_count_volume_in_tissue.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume_in_tissue)))  

In [7]:
# Reading pixels file to count cell activity

In [8]:
regions_count_activity_fracc = pd.read_csv("SMM_s1_DR_Seg_3Dcount_75slices.csv")
regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"] 

In [9]:
# Adding-up regions' fracctions

In [20]:
regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})

total_region_activity = regions_fracc_1.append(regions_fracc_2, ignore_index=True).append(regions_fracc_3, ignore_index=True)
total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)

In [ ]:
# pixels_fracc_1 = pixels_count_activity_fracc[["PIXEL_COUNT", "INTENSITY_1", "INTENSITY_1_PERC"]]
# pixels_fracc_2 = pixels_count_activity_fracc[["PIXEL_COUNT", "INTENSITY_2", "INTENSITY_2_PERC"]]
# pixels_fracc_3 = pixels_count_activity_fracc[["PIXEL_COUNT", "INTENSITY_3", "INTENSITY_3_PERC"]]
# pixels_fracc_1["TOTAL_PIXELS"] = pixels_fracc_1["PIXEL_COUNT"].astype(int) * pixels_fracc_1["INTENSITY_1_PERC"].astype(float)
# pixels_fracc_2["TOTAL_PIXELS"] = pixels_fracc_2["PIXEL_COUNT"].astype(int) * pixels_fracc_2["INTENSITY_2_PERC"].astype(float)
# pixels_fracc_3["TOTAL_PIXELS"] = pixels_fracc_3["PIXEL_COUNT"].astype(int) * pixels_fracc_3["INTENSITY_3_PERC"].astype(float)
# pixels_fracc_1 = pixels_fracc_1[["TOTAL_PIXELS","INTENSITY_1"]]
# pixels_fracc_1 = pixels_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY"})
# pixels_fracc_2 = pixels_fracc_2[["TOTAL_PIXELS","INTENSITY_2"]]
# pixels_fracc_2 = pixels_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY"})
# pixels_fracc_3 = pixels_fracc_3[["TOTAL_PIXELS","INTENSITY_3"]]
# pixels_fracc_3 = pixels_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY"})
# total_pixel_activity = pixels_fracc_1.append(pixels_fracc_2, ignore_index=True).append(pixels_fracc_3, ignore_index=True)
# total_pixel_activity = total_pixel_activity.groupby(['INTENSITY']).agg('sum').reset_index()

In [ ]:
# Adding volumes and activity columns to the template table

In [24]:
output_s1 = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
output_s1 = output_s1.drop(columns=['INTENSITY'])
output_s1 = output_s1.rename(columns = {'PIXEL_COUNT':'VOLUME'})
output_s1 = output_s1.merge(pixels_count_volume_in_tissue, how='left', left_on='LabelID', right_on='INTENSITY')
output_s1['VOLUME'] = output_s1['VOLUME'].div(1000000).round(6)
output_s1 = output_s1.drop(columns=['INTENSITY'])
output_s1 = output_s1.rename(columns = {'PIXEL_COUNT':'VOLUME_IN_TISSUE'})
output_s1['VOLUME_IN_TISSUE'] = output_s1['VOLUME_IN_TISSUE'].div(1000000).round(6)
output_s1["LabelID"] = output_s1["LabelID"].astype(int)
total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)
output_s1 = output_s1.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
output_s1 = output_s1.drop(columns=['INTENSITY'])
output_s1 = output_s1.rename(columns = {'COUNTS':'ACTIVITY'})

In [ ]:
total_pixel_activity

In [21]:
total_region_activity

,INTENSITY,COUNTS
0,0,0.0
1,1,24.0
2,10,2190.0
3,100,307.0
4,1004,60.0
...,...,...
753,988,227.0
754,989,65.0
755,990,69.0
756,996,71.0


In [25]:
output_s1

,LabelID,Region,LabelAbrv,VOLUME,VOLUME_IN_TISSUE,ACTIVITY
0,1,Right Tuberomammillary nucleus ventral part,RTMv,0.005824,0.004211,24.0
1,2,Right Primary somatosensory area mouth layer 6b,RSSp-m6b,0.022375,0.022375,33.0
2,4,Right Inferior colliculus,RIC,1.213398,1.170235,4709.0
3,6,Right internal capsule,Rint,0.518812,0.517995,315.0
4,7,Right Principal sensory nucleus of the trigeminal,RPSV,0.201033,0.200994,234.0
...,...,...,...,...,...,...
1106,21127,Left Temporal association areas layer 2/3,LTEa2/3,0.000000,0.000000,NaN
1107,21128,Left Primary somatosensory area lower limb lay...,LSSp-ll5,0.000000,0.000000,NaN
1108,21139,Left Nucleus of the lateral olfactory tract la...,LNLOT3,0.000000,0.000000,NaN
1109,30703,"Left Dentate gyrus, molecular layer",LDG-mo,0.050217,0.000028,5.0
